In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import psycopg2
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
db_params = {
    'host': 'localhost',
    'database': 'AlbertaWildlifeCollision',
    'user': 'postgres',
    'password': 'rashmi@2024'
}

connection = psycopg2.connect(**db_params)


In [14]:
import pandas as pd
import psycopg2

# List of table names 
table_names = ['csv_2016', 'csv_2017', 'csv_2018','csv_2019','csv_2020','csv_2021']


combined_df = pd.DataFrame()


# Loop through the list of table names
for table_name in table_names:
    query = f"SELECT * FROM {table_name};"
    
    # Fetch data from the table into a DataFrame
    table_df = pd.read_sql_query(query, connection)
    
    # Append the data to the combined DataFrame
    combined_df = pd.concat([combined_df, table_df], ignore_index=True)

# Close the database connection
connection.close()

# Now, 'combined_df' contains the data from all specified tables


C:\Users\rashm\AppData\Local\Temp\ipykernel_16544\3357745728.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_df = pd.read_sql_query(query, connection)


In [15]:
combined_df

,month,year,species,longitude,latitude
0,January,2016,"Deer, White-Tailed",57.147963,-111.587578
1,January,2016,"Deer, White-Tailed",56.800923,-111.435825
2,January,2016,"Deer, White-Tailed",55.172016,-112.480026
3,January,2016,"Deer, White-Tailed",56.392280,-111.443014
4,January,2016,Duck/Goose/Swan,56.711200,-111.385400
...,...,...,...,...,...
23018,December,2020,"Sheep, Bighorn",49.623903,-114.658774
23019,December,2020,"Skunk, Striped",53.570560,-116.476160
23020,December,2020,"Skunk, Striped",49.589707,-114.199564
23021,December,2020,"Skunk, Striped",49.417056,-110.813096


In [16]:
combined_df.isnull().sum()

month        0
year         0
species      0
longitude    0
latitude     0
dtype: int64

In [17]:
combined_df.dropna()

,month,year,species,longitude,latitude
0,January,2016,"Deer, White-Tailed",57.147963,-111.587578
1,January,2016,"Deer, White-Tailed",56.800923,-111.435825
2,January,2016,"Deer, White-Tailed",55.172016,-112.480026
3,January,2016,"Deer, White-Tailed",56.392280,-111.443014
4,January,2016,Duck/Goose/Swan,56.711200,-111.385400
...,...,...,...,...,...
23018,December,2020,"Sheep, Bighorn",49.623903,-114.658774
23019,December,2020,"Skunk, Striped",53.570560,-116.476160
23020,December,2020,"Skunk, Striped",49.589707,-114.199564
23021,December,2020,"Skunk, Striped",49.417056,-110.813096


In [18]:
df=combined_df

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23023 entries, 0 to 23022
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   month      23023 non-null  object 
 1   year       23023 non-null  int64  
 2   species    23023 non-null  object 
 3   longitude  23023 non-null  float64
 4   latitude   23023 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 899.5+ KB


In [20]:
df.dtypes

month         object
year           int64
species       object
longitude    float64
latitude     float64
dtype: object

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23023 entries, 0 to 23022
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   month      23023 non-null  object 
 1   year       23023 non-null  int64  
 2   species    23023 non-null  object 
 3   longitude  23023 non-null  float64
 4   latitude   23023 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 899.5+ KB


In [22]:
df

,month,year,species,longitude,latitude
0,January,2016,"Deer, White-Tailed",57.147963,-111.587578
1,January,2016,"Deer, White-Tailed",56.800923,-111.435825
2,January,2016,"Deer, White-Tailed",55.172016,-112.480026
3,January,2016,"Deer, White-Tailed",56.392280,-111.443014
4,January,2016,Duck/Goose/Swan,56.711200,-111.385400
...,...,...,...,...,...
23018,December,2020,"Sheep, Bighorn",49.623903,-114.658774
23019,December,2020,"Skunk, Striped",53.570560,-116.476160
23020,December,2020,"Skunk, Striped",49.589707,-114.199564
23021,December,2020,"Skunk, Striped",49.417056,-110.813096


In [25]:
combined_df['month'] = combined_df['month'].apply(lambda x: str(x) if x is not None else '')

In [27]:
df=combined_df

In [31]:
# Encoding the 'species' column to numerical values
le = LabelEncoder()
df['species_encoded'] = le.fit_transform(df['species'])

# One-hot encode the 'month' column
df = pd.get_dummies(df, columns=['month'], drop_first=True)

# Define features (X) and target variable (y)
# Assuming you want to use 'year', 'longitude', 'latitude', and one-hot encoded 'month' features
X = df[['year', 'longitude', 'latitude'] + list(df.filter(regex='^month_').columns)]
y = df['species_encoded']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [50]:

# Creating a Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Training the classifier on the training set
clf.fit(X_train, y_train)

# Making predictions on the test set
y_pred = clf.predict(X_test)

# Get unique classes from the target variable in the test set
unique_classes = le.inverse_transform(df['species_encoded'].unique())

# Print classification report for more detailed metrics
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_, labels=le.transform(le.classes_)))



Classification Report:
                           precision    recall  f1-score   support

          Animal, Unknown       0.75      0.41      0.53        22
                   Badger       0.43      0.34      0.38        38
              Bear, Black       0.52      0.39      0.44        31
            Bear, Grizzly       0.00      0.00      0.00         2
                   Beaver       0.44      0.47      0.45        15
                   Bobcat       0.00      0.00      0.00         1
                Bullsnake       1.00      1.00      1.00         2
        Caribou, Woodland       0.00      0.00      0.00         3
            Cat, Domestic       0.58      0.56      0.57        66
                   Cougar       0.00      0.00      0.00         0
                   Coyote       0.55      0.49      0.52       307
               Deer, Mule       0.66      0.68      0.67      1209
            Deer, Unknown       0.49      0.35      0.41       124
       Deer, White-Tailed       0.67  

C:\Users\rashm\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rashm\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rashm\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rashm\anaconda3\Lib\site-packages\sklearn\metrics\_cla

In [43]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.63


In [45]:
feature_importances = clf.feature_importances_
print("Feature Importances:")
for feature, importance in zip(X.columns, feature_importances*100):
    print(f"{feature}: {importance}")


Feature Importances:
year: 5.930856781537298
longitude: 44.188042697803866
latitude: 43.32393719464426
month_August: 0.7100337192564025
month_December: 0.6853552826848476
month_February: 0.40771159680706437
month_January: 0.4505942175598906
month_July: 0.615647675197208
month_June: 0.5585506112643527
month_March: 0.41038134072347887
month_May: 0.5251870738433412
month_November: 0.6878106484709051
month_October: 0.6157923146703101
month_September: 0.8900988455367755


In [62]:
from plotly.express import scatter_geo
scatter_geo(data_frame=combined_df, lat='latitude', lon='longitude',  fitbounds='locations', basemap_visible=True, hover_name='month',
            color='species',
            scope='north america').update_layout(autosize=True, ).show()

In [61]:
combined_df

,month,year,species,longitude,latitude,species_encoded
0,January,2016,"Deer, White-Tailed",57.147963,-111.587578,13
1,January,2016,"Deer, White-Tailed",56.800923,-111.435825,13
2,January,2016,"Deer, White-Tailed",55.172016,-112.480026,13
3,January,2016,"Deer, White-Tailed",56.392280,-111.443014,13
4,January,2016,Duck/Goose/Swan,56.711200,-111.385400,15
...,...,...,...,...,...,...
23018,December,2020,"Sheep, Bighorn",49.623903,-114.658774,38
23019,December,2020,"Skunk, Striped",53.570560,-116.476160,39
23020,December,2020,"Skunk, Striped",49.589707,-114.199564,39
23021,December,2020,"Skunk, Striped",49.417056,-110.813096,39
